In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np 
import sqlite3
import matplotlib.pyplot as plt
from time import sleep
from apikey import API_KEY

In [2]:
watcher = LolWatcher(API_KEY)
region = 'na1'

In [3]:
me = watcher.summoner.by_name(region, "Linom")
me

{'id': 'Qj8ZC0bzUosODd7I5t91jVhRmkQ9zJ5EBKBF7H2Sqq0dIjM',
 'accountId': 'SH72nzFnzQq72Nv9OUasiU7iO8xGY3s6fp2Q5EC0BID45Vs',
 'puuid': 'pXozCpOspMGJvjixjONFRy6dBG9bj5rt4KeJOJCr1wAGprTEDaNOX3OTFgG6o8RK1QpFjihDLiIVxg',
 'name': 'Linom',
 'profileIconId': 1594,
 'revisionDate': 1642980665054,
 'summonerLevel': 298}

In [4]:
challenger_dto = watcher.league.challenger_by_queue(region, "RANKED_SOLO_5x5")
challenger_players = challenger_dto["entries"]
challenger_players

[{'summonerId': 'u5c64MfzXcu15zjhuwjVhcjoMLTBd2AoMPsK44V6OPEbtqQ',
  'summonerName': 'Ariendel',
  'leaguePoints': 668,
  'rank': 'I',
  'wins': 79,
  'losses': 67,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': False},
 {'summonerId': 'pYphlOrM4FJSyLjvz06GasxO87dpcceqG4lBLlAiGIVOI4hT',
  'summonerName': 'DC EGC SOL',
  'leaguePoints': 638,
  'rank': 'I',
  'wins': 102,
  'losses': 77,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': False},
 {'summonerId': 'KvY92Iil1IsoxtZAR5DpSREUJt89wNGSOS64WR_lYZbBiSw',
  'summonerName': 'Tojí Fushiguro',
  'leaguePoints': 555,
  'rank': 'I',
  'wins': 74,
  'losses': 49,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': False},
 {'summonerId': 'FZ-hOcROICnvto-T5rSuFNJHyxGvUGeeJVfS5sXhzip2Rxk',
  'summonerName': 'KingClueless',
  'leaguePoints': 550,
  'rank': 'I',
  'wins': 76,
  'losses': 60,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'ho

Grab dragons, hearalds, baron per team

check if len(game_1["info"]["frames"]) > 22

In [5]:
match_list = watcher.match.matchlist_by_puuid('americas', me["puuid"], type="ranked")
match_detail = watcher.match.by_id('americas', match_list[0])
game_1 = watcher.match.timeline_by_match('americas', match_list[0])
game_1["info"]["frames"]

blue_dragons = 0
blue_heralds = 0
red_dragons = 0
red_heralds = 0
blue_barons = 0
red_barons = 0

for frame in game_1["info"]["frames"]:
    for event in frame["events"]:
        if "monsterType" in event.keys():
            if event["monsterType"] == "DRAGON":
                if event["killerTeamId"] == 100:
                    blue_dragons += 1
                else:
                    red_dragons += 1
            if event["monsterType"] == "RIFTHERALD":
                if event["killerTeamId"] == 100:
                    blue_heralds += 1
                else:
                    red_heralds += 1
            if event["monsterType"] == "BARON_NASHOR":
                if event["killerTeamId"] == 100:
                    blue_barons += 1
                else:
                    red_barons += 1
            
print(blue_dragons, red_dragons, blue_heralds, red_heralds, blue_barons, red_barons)

0 0 0 0 0 0


Grab kills per team

In [6]:
blue_k = 0 # blue side (bottom)
red_k = 0 # red side (top)

for frame in game_1["info"]["frames"]:
    for event in frame["events"]:
        if "victimId" in event.keys():
            if event["type"] == "CHAMPION_KILL":
                if event["killerId"] == 0:
                    continue
                if 1 <= event["killerId"] <= 5:
                    blue_k += 1
                else:
                    red_k += 1
print(blue_k, red_k)


13 4


Grab obj per team

In [8]:
blue_obj = 0
red_obj = 0
for frame in game_1["info"]["frames"]:
    for event in frame["events"]:
        if "buildingType" in event.keys():
            if event["teamId"] == 100:
                red_obj += 1
            else:
                blue_obj += 1
            # print(event)
print(blue_obj, red_obj)

0 0


Grab gold per team

In [20]:
if match_detail["info"]["participants"][0]["win"] == True:
    winner = "Team 1"
else:
    winner = "Team 2"
print(winner)
# team_1_win = game_details["info"]["participants"][0]["win"]


if len(game_timeline['info']['frames']) < 22:
    break 
        
frames =  game_timeline['info']['frames'][:22]

gold = {}
for k in range(len(frames)):
    gold['{}'.format(k)] = {}
for value in gold.values():
    for l in range(1, 11):
        value['{}'.format(l)] = 0

for m in range(len(frames)):
    for n in range(1, 11):
        gold['{}'.format(m)]['{}'.format(n)] = game_1['info']['frames'][m]['participantFrames']['{}'.format(n)]['totalGold']

blue_gold = []
red_gold = []
for key in gold.keys():
    team1 = 0
    team2 = 0
    for o in range(1, 6):
        team1 += gold[key]['{}'.format(o)]
        team2 += gold[key]['{}'.format(o + 5)]
    blue_gold.append(team1)
    red_gold.append(team2)
frame_size = [val for val in range(len(blue_gold))]

gold_diff = (np.array(blue_gold) - np.array(red_gold)).tolist()

# id, game_id, team_1, team_2, gold_diff, winner
# cur.execute(insert_match, (match_list[i], str(team_1_gold), str(team_2_gold), str(gold_diff), winner))
# connection.commit()
# sleep(0.85)
print(blue_gold, red_gold)

SyntaxError: 'break' outside loop (Temp/ipykernel_3184/3599083771.py, line 13)